In [2]:
from joblib import Parallel, delayed

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import matplotlib.pyplot as plt
import numpy as np
from gensim.models.ldamulticore import LdaMulticore
import pandas as pd 
import pyLDAvis
import pyLDAvis.gensim


# Tokeniza cada texto en una lista de palabras
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations



In [42]:
#dataset_name = sys.argv[1]
#df = pd.read_csv(dataset_name)
df = pd.read_csv("datosProcesados.csv")


# Seleccionar las filas en las que la opinion sea negativa, mas tarde podemos probar con las positivas y las neutrales
X = df[df['__target__'] == 0]
X = X[X['airline'] == 'US Airways']

# Paso todos los textos a una lista
textos = X.text.values.tolist()

# Paso cada texto de cada tweet de una string a una lista de palabras
data_words = list(sent_to_words(textos))

# Se crea el diccionario de las palabras; cada palabra unica contiene un identificador. Sirve para crear el corpus
id2word = corpora.Dictionary(data_words)

# Se crea el corpus
corpus = [id2word.doc2bow(text) for text in data_words]
#print(corpus[0])
# Cada palabra: (word_id, word_frequency). Si es (47,3) quiere decir que la palabra con id 47 aparece 3 veces en el documento



In [43]:
X

Unnamed: 0      tweet_id  airline_sentiment_confidence   
8966         8966  5.703106e+17                        0.6292  \
8967         8967  5.703101e+17                        1.0000   
8970         8970  5.703088e+17                        1.0000   
8972         8972  5.703076e+17                        1.0000   
8973         8973  5.703071e+17                        0.7020   
...           ...           ...                           ...   
11873       11873  5.677126e+17                        0.6716   
11874       11874  5.677102e+17                        1.0000   
11875       11875  5.676980e+17                        1.0000   
11876       11876  5.676795e+17                        1.0000   
11877       11877  5.676710e+17                        1.0000   

                    negativereason  negativereason_confidence     airline   
8966       Flight Booking Problems                     0.3146  US Airways  \
8967        Customer Service Issue                     1.0000  US Airways   
8970        Customer Service Issue                     0.6452  US Airways   
8972        Customer Service Issue                     1.0000  US Airways   
8973   Flight Attendant Complaints                     0.7020  US Airways   
...                            ...                        ...         ...   
11873                  Late Flight                     0.6716  US Airways   
11874       Customer Service Issue                     0.6838  US Airways   
11875       Customer Service Issue                     0.6963  US Airways   
11876       Customer Service Issue                     0.7188  US Airways   
11877       Customer Service Issue                     1.0000  US Airways   

                 name  retweet_count   
8966        jhazelnut            0.0  \
8967         GAKotsch            0.0   
8970    retardedlarry            0.0   
8972    Matt_Bernanke            0.0   
8973   jeremyleewhite            0.0   
...               ...            ...   
11873        sankeshw            0.0   
11874  CharNewsJunkie            0.0   
11875     MarkKersten            0.0   
11876     DonnyYardas            0.0   
11877      sevnthstar            0.0   

                                                    text   
8966   usairway better time call flight friday need c...  \
8967                      usairway one agent avail speak   
8970   usairway hung anoth wast hour time suppos book...   
8972                            usairway your kill insid   
8973   usairway new americanair like new spiritairlin...   
...                                                  ...   
11873  usairway 2pm flight fll phl connect man howev ...   
11874  usairway hold gold reserv line 3 hour flight c...   
11875  usairway choic pay anoth 50 go airport hope tr...   
11876             usairway reserv hold 2 hour hang upsmh   
11877  usairway americanair r u suppos chang flight u...   

                       tweet_coord              tweet_created   
8966      [46.3144754, 11.0480288]  2015-02-24 11:53:37 -0800  \
8967     [45.1960403, -63.1653789]  2015-02-24 11:51:48 -0800   
8970      [46.3144754, 11.0480288]  2015-02-24 11:46:28 -0800   
8972     [-0.2201641, -78.5123274]  2015-02-24 11:41:43 -0800   
8973   [35.21979387, -80.94498281]  2015-02-24 11:39:45 -0800   
...                            ...                        ...   
11873     [33.5945144, -7.6200284]  2015-02-17 07:50:06 -0800   
11874    [40.7587979, -73.9623427]  2015-02-17 07:40:44 -0800   
11875    [-0.2201641, -78.5123274]  2015-02-17 06:52:12 -0800   
11876    [40.7587979, -73.9623427]  2015-02-17 05:38:31 -0800   
11877    [37.7790262, -122.419906]  2015-02-17 05:04:44 -0800   

                  tweet_location               user_timezone  __target__  
8966                         NaN                         NaN           0  
8967         Nova Scotia, Canada      Atlantic Time (Canada)           0  
8970                         NaN                         NaN           0  
8972              

In [36]:
lda_model = LdaMulticore(corpus=corpus,  
    id2word=id2word,
    num_topics=5, 
    random_state=100,
    chunksize=100,
    passes=20,
    iterations=100,
    eval_every= None,
    per_word_topics=True,
    workers=8)
    
# La idea es tener la mejor similitud entre los textos de un topico mientras que se minimiza la similitud con los textos de otros
# La coherencia relaciona la distancia intracluster con la distancia intercluster
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_words, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()


# Imprime los topicos; por cada topico muestra su id y luego las palabras mas frecuentes con la frecuencia de esa palabra en ese topico
print(lda_model.print_topics())


[(0, '0.038*"jetblu" + 0.031*"unit" + 0.022*"servic" + 0.020*"custom" + 0.017*"bag" + 0.014*"wait" + 0.013*"plane" + 0.012*"gate" + 0.009*"southwestair" + 0.009*"delay"'), (1, '0.056*"usairway" + 0.018*"unit" + 0.016*"flight" + 0.012*"americanair" + 0.012*"seat" + 0.009*"jetblu" + 0.009*"day" + 0.007*"help" + 0.007*"book" + 0.007*"ticket"'), (2, '0.130*"flight" + 0.043*"cancel" + 0.033*"delay" + 0.028*"unit" + 0.021*"flightl" + 0.020*"hour" + 0.017*"late" + 0.016*"miss" + 0.013*"connect" + 0.011*"hr"'), (3, '0.043*"southwestair" + 0.030*"unit" + 0.022*"fli" + 0.019*"airlin" + 0.019*"never" + 0.016*"custom" + 0.015*"worst" + 0.013*"ever" + 0.013*"get" + 0.013*"servic"'), (4, '0.133*"usairway" + 0.032*"hold" + 0.022*"hour" + 0.021*"get" + 0.020*"call" + 0.017*"phone" + 0.015*"flight" + 0.015*"help" + 0.014*"southwestair" + 0.013*"tri"')]


In [39]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds="mmds", R=30)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.181629 -0.030398       1        1  25.265266
2      0.123377  0.223919       2        1  20.321598
1     -0.169834 -0.097511       3        1  20.066439
4     -0.200640  0.147319       4        1  18.708804
3      0.065468 -0.243329       5        1  15.637892, topic_info=              Term         Freq        Total Category  logprob  loglift
2691      usairway  2868.000000  2868.000000  Default  30.0000  30.0000
16          flight  2736.000000  2736.000000  Default  29.0000  29.0000
192         cancel   688.000000   688.000000  Default  28.0000  28.0000
181         jetblu   917.000000   917.000000  Default  27.0000  27.0000
183   southwestair   931.000000   931.000000  Default  26.0000  26.0000
...            ...          ...          ...      ...      ...      ...
415           make    51.990542   256.598154   Topic5  -5.4757   0.2590
188          delay    59.493586   760.220824   Topic5  -5.3409  -0.6923
422            day    53.429337   345.646572   Topic5  -5.4484  -0.0116
350           dont    51.403114   292.838080   Topic5  -5.4870   0.1156
1030   americanair    50.629372   459.761689   Topic5  -5.5022  -0.3507

[364 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
2031      5  0.995854        aa
66        1  0.218907       abl
66        4  0.744285       abl
66        5  0.021891       abl
1616      5  0.955029  accommod
...     ...       ...       ...
22        3  0.294955     would
22        4  0.083192     would
22        5  0.166385     would
1041      5  0.975575      yeah
1106      5  0.979922      youv

[614 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 3, 2, 5, 4])